# Project: Wrangle WeRateDogs data from Twitter

Table of Contents
1. Introduction
2. Data Wrangling
3. Storing and Acting on Wrangling Data

1.Introduction
This project is about wrangling data for the Wrangle and Analyze Project
The document is divided in 3 sections, each for one step of the dta wrangling process that we saw in the lessons

Context
Our goal: wrangle WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. The Twitter archive is great, but it only contains very basic tweet information. Additional gathering, then assessing and cleaning is required for "Wow!"-worthy analyses and visualizations.

Project Details
Our tasks in this project are as follows:

Data wrangling, which consists of:
Gathering data (downloadable file in the Resources tab in the left most panel of your classroom and linked in step 1 below).
Assessing data
Cleaning data
Storing, analyzing, and visualizing your wrangled data
Reporting on 1) your data wrangling efforts and 2) your data analyses and visualizations


2. Data Wrangling

Gather
First we need to gather the data from at least 3 different sources. Some data were wprovides and other were obtained

In [5]:
#first load all the libraries needed

import pandas as pd
import numpy as np
import requests
import json
import tweepy
import os
import time
import re
import matplotlib.pyplot as plt
import warnings
from IPython.display import Image
from functools import reduce
import re
import seaborn as sns
import datetime
%matplotlib inline

In [7]:
#Now load the twitter_archive_enhanced.csv file into the dataframe
tweet_data = pd.read_csv("twitter-archive-enhanced.csv")
tweet_data.head()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com/download/iphone" r...   
3  <a href="http://twitter.com/download/iphone" r...   
4  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   
2  This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3  This is Darla. She commenced a snooze mid meal...                  NaN   
4  This is Franklin. He would like you to stop ca...                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   
3                       NaN                        NaN   
4                       NaN                        NaN   

                                       expanded_urls  rating_numerator  \
0  https://twitter.com/dog_rates/status/892420643...                13   
1  https://twitter.com/dog_rates/status/892177421...                13   
2  https://twitter.com/dog_rates/status/891815181...                12   
3  https://twitter.com/dog_rates/status/891689557...                13   
4  https://twitter.com/dog_rates/status/891327558...                12   

   rating_denominator      name doggo floofer pupper puppo  
0                  10   Phineas  None    None   None  None  
1                  10     Tilly  None    None   None  None  
2                  10    Archie  None    None   None  None  
3                  10     Darla  None    None   None  None  
4                  10  Franklin  None    None   None  None

In [8]:
# Now, we retrieve image-predictions.tsv file
prediction_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
with open('image-predictions.tsv', 'wb') as prediction_file:
    prediction_gather = requests.get(prediction_url)
    prediction_file.write(prediction_gather.content)
    
#Now we retrieve the image_predictions.tsv file into its dataframe
prediction_data = pd.read_csv("image_predictions.tsv",sep='\t')
prediction_data.head()

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   
3        1     Rhodesian_ridgeback  0.408143    True             redbone   
4        1      miniature_pinscher  0.560311    True          Rottweiler   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True  
2  0.138584    True           bloodhound  0.116197    True  
3  0.360687    True   miniature_pinscher  0.222752    True  
4  0.243682    True             Doberman  0.154629    True

In [11]:
# Now we load the twtter data file
consumer_key = 'XXX'
consumer_secret = 'XXX'
access_token = 'XXX'
access_secret = 'XXX'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)


In [20]:
tweet_json = pd.read_json("tweet-json.txt", lines=True)

tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   created_at                     2354 non-null   datetime64[ns, UTC]
 1   id                             2354 non-null   int64              
 2   id_str                         2354 non-null   int64              
 3   full_text                      2354 non-null   object             
 4   truncated                      2354 non-null   bool               
 5   display_text_range             2354 non-null   object             
 6   entities                       2354 non-null   object             
 7   extended_entities              2073 non-null   object             
 8   source                         2354 non-null   object             
 9   in_reply_to_status_id          78 non-null     float64            
 10  in_reply_to_status_id_st

Assess

Assessing our data is the second step in data wrangling. When assessing, we are like a detective at work, inspecting our dataset for two things: data quality issues (i.e. content issues) and lack of tidiness (i.e. structural issues).

Quality issues are issues with content, like inaccurate or duplicate data. Tidiness issues are structural issues, specifically: each variable must be a column, each observation must be a row, and each type of observational unit must be a table.

We will use two steps: Visual Assessment and Programmatic Assessment

1. Dataframe contents for visual assessment

In [4]:
# Display the twitter data dataframe
tweet_data

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0     892420643555336193                    NaN                  NaN   
1     892177421306343426                    NaN                  NaN   
2     891815181378084864                    NaN                  NaN   
3     891689557279858688                    NaN                  NaN   
4     891327558926688256                    NaN                  NaN   
5     891087950875897856                    NaN                  NaN   
6     890971913173991426                    NaN                  NaN   
7     890729181411237888                    NaN                  NaN   
8     890609185150312448                    NaN                  NaN   
9     890240255349198849                    NaN                  NaN   
10    890006608113172480                    NaN                  NaN   
11    889880896479866881                    NaN                  NaN   
12    889665388333682689                    NaN                  NaN   
13    889638837579907072                    NaN                  NaN   
14    889531135344209921                    NaN                  NaN   
15    889278841981685760                    NaN                  NaN   
16    888917238123831296                    NaN                  NaN   
17    888804989199671297                    NaN                  NaN   
18    888554962724278272                    NaN                  NaN   
19    888202515573088257                    NaN                  NaN   
20    888078434458587136                    NaN                  NaN   
21    887705289381826560                    NaN                  NaN   
22    887517139158093824                    NaN                  NaN   
23    887473957103951883                    NaN                  NaN   
24    887343217045368832                    NaN                  NaN   
25    887101392804085760                    NaN                  NaN   
26    886983233522544640                    NaN                  NaN   
27    886736880519319552                    NaN                  NaN   
28    886680336477933568                    NaN                  NaN   
29    886366144734445568                    NaN                  NaN   
...                  ...                    ...                  ...   
2326  666411507551481857                    NaN                  NaN   
2327  666407126856765440                    NaN                  NaN   
2328  666396247373291520                    NaN                  NaN   
2329  666373753744588802                    NaN                  NaN   
2330  666362758909284353                    NaN                  NaN   
2331  666353288456101888                    NaN                  NaN   
2332  666345417576210432                    NaN                  NaN   
2333  666337882303524864                    NaN                  NaN   
2334  666293911632134144                    NaN                  NaN   
2335  666287406224695296                    NaN                  NaN   
2336  666273097616637952                    NaN                  NaN   
2337  666268910803644416                    NaN                  NaN   
2338  666104133288665088                    NaN                  NaN   
2339  666102155909144576                    NaN                  NaN   
2340  666099513787052032                    NaN                  NaN   
2341  666094000022159362                    NaN                  NaN   
2342  666082916733198337                    NaN                  NaN   
2343  666073100786774016                    NaN                  NaN   
2344  666071193221509120                    NaN                  NaN   
2345  666063827256086533                    NaN                  NaN   
2346  666058600524156928                    NaN                  NaN   
2347  666057090499244032                    NaN                  NaN   
2348  666055525042405380                    NaN                  NaN   
2349  666051853826850816                    NaN         

Visual
Quality Issues
1. Name column, have "None" as name. This can be confusing
2. There is missing data in "name" columns and is written as the string "None"

Tidity Issues
1. The columns "doggo", "floofer", "pupper" and "puppo" represent the same variable "type"

In [5]:
# Display the prediction data dataframe
prediction_data

tweet_id                                            jpg_url  \
0     666020888022790149    https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1     666029285002620928    https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2     666033412701032449    https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3     666044226329800704    https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4     666049248165822465    https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   
5     666050758794694657    https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg   
6     666051853826850816    https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg   
7     666055525042405380    https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg   
8     666057090499244032    https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg   
9     666058600524156928    https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg   
10    666063827256086533    https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg   
11    666071193221509120    https://pbs.twimg.com/media/CT5cN_3WEAAlOoZ.jpg   
12    666073100786774016    https://pbs.twimg.com/media/CT5d9DZXAAALcwe.jpg   
13    666082916733198337    https://pbs.twimg.com/media/CT5m4VGWEAAtKc8.jpg   
14    666094000022159362    https://pbs.twimg.com/media/CT5w9gUW4AAsBNN.jpg   
15    666099513787052032    https://pbs.twimg.com/media/CT51-JJUEAA6hV8.jpg   
16    666102155909144576    https://pbs.twimg.com/media/CT54YGiWUAEZnoK.jpg   
17    666104133288665088    https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg   
18    666268910803644416    https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg   
19    666273097616637952    https://pbs.twimg.com/media/CT8T1mtUwAA3aqm.jpg   
20    666287406224695296    https://pbs.twimg.com/media/CT8g3BpUEAAuFjg.jpg   
21    666293911632134144    https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg   
22    666337882303524864    https://pbs.twimg.com/media/CT9OwFIWEAMuRje.jpg   
23    666345417576210432    https://pbs.twimg.com/media/CT9Vn7PWoAA_ZCM.jpg   
24    666353288456101888    https://pbs.twimg.com/media/CT9cx0tUEAAhNN_.jpg   
25    666362758909284353    https://pbs.twimg.com/media/CT9lXGsUcAAyUFt.jpg   
26    666373753744588802    https://pbs.twimg.com/media/CT9vZEYWUAAlZ05.jpg   
27    666396247373291520    https://pbs.twimg.com/media/CT-D2ZHWIAA3gK1.jpg   
28    666407126856765440    https://pbs.twimg.com/media/CT-NvwmW4AAugGZ.jpg   
29    666411507551481857    https://pbs.twimg.com/media/CT-RugiWIAELEaq.jpg   
...                  ...                                                ...   
2045  886366144734445568    https://pbs.twimg.com/media/DE0BTnQUwAApKEH.jpg   
2046  886680336477933568    https://pbs.twimg.com/media/DE4fEDzWAAAyHMM.jpg   
2047  886736880519319552    https://pbs.twimg.com/media/DE5Se8FXcAAJFx4.jpg   
2048  886983233522544640    https://pbs.twimg.com/media/DE8yicJW0AAAvBJ.jpg   
2049  887101392804085760    https://pbs.twimg.com/media/DE-eAq6UwAA-jaE.jpg   
2050  887343217045368832  https://pbs.twimg.com/ext_tw_video_thumb/88734...   
2051  887473957103951883    https://pbs.twimg.com/media/DFDw2tyUQAAAFke.jpg   
2052  887517139158093824  https://pbs.twimg.com/ext_tw_video_thumb/88751...   
2053  887705289381826560    https://pbs.twimg.com/media/DFHDQBbXgAEqY7t.jpg   
2054  888078434458587136    https://pbs.twimg.com/media/DFMWn56WsAAkA7B.jpg   
2055  888202515573088257    https://pbs.twimg.com/media/DFDw2tyUQAAAFke.jpg   
2056  888554962724278272    https://pbs.twimg.com/media/DFTH_O-UQAACu20.jpg   
2057  888804989199671297    https://pbs.twimg.com/media/DFWra-3VYAA2piG.jpg   
2058  888917238123831296    https://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg   
2059  889278841981685760  https://pbs.twimg.com/ext_tw_video_thumb/88927...   
2060  889531135344209921    https://pbs.twimg.com/media/DFg_2PVW0AEHN3p.jpg   
2061  889638837579907072    https://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg   
2062  889665388333682689    https://pbs.twimg.com/media/DFi579UWsAAatzw.jpg   
2063  889880896479866881    https://pbs.twimg.com/media/DFl99B1WsA

Visual Quality Issues

3. In p1, p2, and p3 the type of dogs uses underscores that are diffucult to read

Tidity Issues

2. Correct predictions of dogs should be combined with tweet_data

2. Dataframe contents for programmatic assessment

In [6]:
#We display the structure of the dataframe tweet_data
tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

Quality Issues
4. Missing "expanded URLS", there are 2297 instead of 2356
5. retweeted_status_id need to be "int64"
6. timestamp is better to be "date_time" objet type
7. there are retweet data, it shouldn´t

In [7]:
#look for more issues with sample rows
tweet_data.sample(15)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
1063  741067306818797568                    NaN                  NaN   
62    880095782870896641                    NaN                  NaN   
2047  671518598289059840                    NaN                  NaN   
1799  677187300187611136                    NaN                  NaN   
1690  681339448655802368                    NaN                  NaN   
927   754856583969079297                    NaN                  NaN   
501   813096984823349248                    NaN                  NaN   
157   861288531465048066                    NaN                  NaN   
121   869596645499047938                    NaN                  NaN   
2214  668544745690562560                    NaN                  NaN   
315   835152434251116546                    NaN                  NaN   
2207  668627278264475648                    NaN                  NaN   
1775  678021115718029313                    NaN                  NaN   
969   750132105863102464                    NaN                  NaN   
813   771102124360998913                    NaN                  NaN   

                      timestamp  \
1063  2016-06-10 00:39:48 +0000   
62    2017-06-28 16:09:20 +0000   
2047  2015-12-01 02:38:04 +0000   
1799  2015-12-16 18:03:28 +0000   
1690  2015-12-28 05:02:37 +0000   
927   2016-07-18 01:53:28 +0000   
501   2016-12-25 19:00:02 +0000   
157   2017-05-07 18:36:02 +0000   
121   2017-05-30 16:49:31 +0000   
2214  2015-11-22 21:41:02 +0000   
315   2017-02-24 15:40:31 +0000   
2207  2015-11-23 03:09:00 +0000   
1775  2015-12-19 01:16:45 +0000   
969   2016-07-05 01:00:05 +0000   
813   2016-08-31 21:47:27 +0000   

                                                 source  \
1063  <a href="http://twitter.com/download/iphone" r...   
62    <a href="http://twitter.com/download/iphone" r...   
2047  <a href="http://twitter.com/download/iphone" r...   
1799  <a href="http://twitter.com/download/iphone" r...   
1690  <a href="http://twitter.com/download/iphone" r...   
927   <a href="http://twitter.com/download/iphone" r...   
501   <a href="http://twitter.com/download/iphone" r...   
157   <a href="http://twitter.com/download/iphone" r...   
121   <a href="http://twitter.com/download/iphone" r...   
2214  <a href="http://twitter.com/download/iphone" r...   
315   <a href="http://twitter.com/download/iphone" r...   
2207  <a href="http://twitter.com/download/iphone" r...   
1775  <a href="http://twitter.com/download/iphone" r...   
969   <a href="http://twitter.com/download/iphone" r...   
813   <a href="http://twitter.com/download/iphone" r...   

                                                   text  retweeted_status_id  \
1063  This is just downright precious af. 12/10 for ...                  NaN   
62    Please don't send in photos without dogs in th...                  NaN   
2047  This is Scruffers. He's being violated on mult...                  NaN   
1799  Here we see a Byzantine Rigatoni. Very aerodyn...                  NaN   
1690  This is Asher. He's not wearing a seatbelt or ...                  NaN   
927   This is Winnie. She's not a fan of the fast mo...                  NaN   
501   This is Rocky. He got triple-doggo-dared. Stuc...                  NaN   
157   HI. MY. NAME. IS. BOOMER. AND. I. WANT. TO. SA...                  NaN   
121   This is Scout. He just graduated. Officially a...                  NaN   
2214  It is an honor to rate this pup. He is a Snork...                  NaN   
315   When you're so blinded by your systematic plag...                  NaN   
2207  This is Timofy. He's a pilot for Southwest. It...                  NaN   
1775  This is Reese. He likes holding hands. 12/10 h...                  NaN   
969   This is Stewie. He will roundhouse kick anyone...                  NaN   
813   This is Charlie. He works for @TODAYshow. Supe...                  NaN   

      retweeted_status_user_id retweeted_status_timestamp  \
1063           

In [8]:
# Look for quality issues in the range of data
tweet_data[['rating_numerator', 'rating_denominator']].describe()

rating_numerator  rating_denominator
count       2356.000000         2356.000000
mean          13.126486           10.455433
std           45.876648            6.745237
min            0.000000            0.000000
25%           10.000000           10.000000
50%           11.000000           10.000000
75%           12.000000           10.000000
max         1776.000000          170.000000

Quality Issues
8. There is a minimum value of 0 in the rating_denominator column
9. The maximun value is higher for both numerator and denominator

In [9]:
#Get the value of the numerator column
tweet_data.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [10]:
#Dig more for the denominator equals zero
tweet_data.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

There is only 1 case that the denominator is "0"

In [11]:
#Looking for duplicated tweet_id
tweet_data.tweet_id.value_counts()

749075273010798592    1
741099773336379392    1
798644042770751489    1
825120256414846976    1
769212283578875904    1
700462010979500032    1
780858289093574656    1
699775878809702401    1
880095782870896641    1
760521673607086080    1
776477788987613185    1
691820333922455552    1
715696743237730304    1
714606013974974464    1
760539183865880579    1
813157409116065792    1
676430933382295552    1
743510151680958465    1
837012587749474308    1
833722901757046785    1
818259473185828864    1
670704688707301377    1
667160273090932737    1
674394782723014656    1
672082170312290304    1
670093938074779648    1
759923798737051648    1
809920764300447744    1
805487436403003392    1
838085839343206401    1
                     ..
763956972077010945    1
870308999962521604    1
720775346191278080    1
785927819176054784    1
783347506784731136    1
775733305207554048    1
834209720923721728    1
825026590719483904    1
758405701903519748    1
668986018524233728    1
6909388994772213

No duplicate tweet_id

In [12]:
# Display the structure of prediction data dataframe
prediction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


Quality Issues
10. There are 2075 entries and in the tweet_data dataframe there are 2376 rows, so there is missing data

In [13]:
#look for duplicate data in prediction_data dataframe
prediction_data.tweet_id.value_counts()

685532292383666176    1
826598365270007810    1
692158366030913536    1
714606013974974464    1
715696743237730304    1
776477788987613185    1
772114945936949249    1
699775878809702401    1
780858289093574656    1
700462010979500032    1
732726085725589504    1
738883359779196928    1
798644042770751489    1
743510151680958465    1
837012587749474308    1
833722901757046785    1
668620235289837568    1
842765311967449089    1
685315239903100929    1
673686845050527744    1
680473011644985345    1
666051853826850816    1
675853064436391936    1
693231807727280129    1
705475953783398401    1
829449946868879360    1
759923798737051648    1
667160273090932737    1
680934982542561280    1
743545585370791937    1
                     ..
794926597468000259    1
776113305656188928    1
825026590719483904    1
834209720923721728    1
775733305207554048    1
669564461267722241    1
879492040517615616    1
720775346191278080    1
666362758909284353    1
750506206503038976    1
6931556864910008

No duplicate data

In [14]:
# Compute statistics on the p*_conf columns in prediction_data
prediction_data[['img_num', 'p1_conf', 'p2_conf', 'p3_conf']].describe()

img_num      p1_conf       p2_conf       p3_conf
count  2075.000000  2075.000000  2.075000e+03  2.075000e+03
mean      1.203855     0.594548  1.345886e-01  6.032417e-02
std       0.561875     0.271174  1.006657e-01  5.090593e-02
min       1.000000     0.044333  1.011300e-08  1.740170e-10
25%       1.000000     0.364412  5.388625e-02  1.622240e-02
50%       1.000000     0.588230  1.181810e-01  4.944380e-02
75%       1.000000     0.843855  1.955655e-01  9.180755e-02
max       4.000000     1.000000  4.880140e-01  2.734190e-01

No quality Issues, no negative numbers, , range between [0,1]

Summary of Quality and Tidiness Issues

Quality
1. Name column, have "None" as name. This can be confusing
2. There is missing data in "name" columns and is written as the string "None"
4. Missing "expanded URLS", there are 2297 instead of 2356
5. retweeted_status_id need to be "int64"
6. timestamp is better to be "date_time" objet type
7. there are retweet data, it shouldn´t
8. There is a minimum value of 0 in the rating_denominator column
9. The maximun value is higher for both numerator and denominator
3. In p1, p2, and p3 the type of dogs uses underscores that are diffucult to read
10. There are 2075 entries and in the tweet_data dataframe there are 2376 rows, so there is missing data

Tidiness
1. The columns "doggo", "floofer", "pupper" and "puppo" represent the same variable "type"
2. Correct predictions of dogs should be combined with tweet_data

Clean

Cleaning our data is the third step in data wrangling. It is where we fix the quality and tidiness issues that we identified in the assess step

We will use the data cleaning process: defining, coding, and testing
•	We will address the missing data first
•	We will tackle the tidiness issues next
•	And finally, we will clean up the quality issues

The very first thing to do before any cleaning occurs is to make a copy of each piece of data. All of the cleaning operations will be conducted on this copy so we can still view the original dirty and/or messy dataset later

In [35]:
#create a copy of each dataframe and merge
tweet_data_clean = tweet_data.copy()
prediction_data_clean = prediction_data.copy()
tweet_json_clean = tweet_json.copy()

# Ref: https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes/44338256
dfs = pd.concat(tweet_data_clean, prediction_data_clean, tweet_json_clean], join='outer', axis=1)
dfs.head()

SyntaxError: invalid syntax (<ipython-input-35-12b673d4909e>, line 7)

Quality

6. timestamp is better to be "date_time" objet type

Define
Convert the timestamp column into datetime

Code

In [16]:
#Changetimestamp column to datetime type
ts_col_names = [t_timestamp for t_timestamp in tweet_data_clean.columns.tolist() if 'timestamp' in t_timestamp]
for ts_col_name in ts_col_names:
    tweet_data_clean[ts_col_name] = pd.to_datetime(tweet_data_clean[ts_col_name])

Test

In [17]:
tweet_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime

7. there are retweet data, it shouldn´t

Define
Remove all rows where retweeted_status_id. Afterwards, reset the index.

Code

In [18]:
retweeted_cols = ['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp']
retweeted_mask = tweet_data_clean.retweeted_status_id.notnull()

tweet_data_clean = tweet_data_clean[~retweeted_mask]
tweet_data_retweet_test = tweet_data_clean.copy() # For testing
tweet_data_clean.drop(columns=retweeted_cols, inplace=True)
tweet_data_clean.reset_index(drop=True, inplace=True)

Test

In [19]:
# chrck if we deleted all of the retweets
tweet_data_retweet_test.retweeted_status_id.notnull().sum()

0

1. Name column, have "None" as name. This can be confusing

Define
We will replace the string "None" with "NaN"

Code

In [20]:
tweet_data_clean.name = tweet_data_clean.name.replace('None', np.nan)

test

In [21]:
tweet_data_clean.name.loc[tweet_data_clean.name == 'None'].sum()

False

In [36]:
# Tidy issue # 1
# Code and test
pd.melt(
    tweet_data_clean, 
    id_vars = ['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp', 'source','text','expanded_urls', 'rating_numerator','rating_denominator','name'],
    var_name ='stages',
    value_name = 'val')

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0      892420643555336193                    NaN                  NaN   
1      892177421306343426                    NaN                  NaN   
2      891815181378084864                    NaN                  NaN   
3      891689557279858688                    NaN                  NaN   
4      891327558926688256                    NaN                  NaN   
5      891087950875897856                    NaN                  NaN   
6      890971913173991426                    NaN                  NaN   
7      890729181411237888                    NaN                  NaN   
8      890609185150312448                    NaN                  NaN   
9      890240255349198849                    NaN                  NaN   
10     890006608113172480                    NaN                  NaN   
11     889880896479866881                    NaN                  NaN   
12     889665388333682689                    NaN                  NaN   
13     889638837579907072                    NaN                  NaN   
14     889531135344209921                    NaN                  NaN   
15     889278841981685760                    NaN                  NaN   
16     888917238123831296                    NaN                  NaN   
17     888804989199671297                    NaN                  NaN   
18     888554962724278272                    NaN                  NaN   
19     888202515573088257                    NaN                  NaN   
20     888078434458587136                    NaN                  NaN   
21     887705289381826560                    NaN                  NaN   
22     887517139158093824                    NaN                  NaN   
23     887473957103951883                    NaN                  NaN   
24     887343217045368832                    NaN                  NaN   
25     887101392804085760                    NaN                  NaN   
26     886983233522544640                    NaN                  NaN   
27     886736880519319552                    NaN                  NaN   
28     886680336477933568                    NaN                  NaN   
29     886366144734445568                    NaN                  NaN   
...                   ...                    ...                  ...   
16462  666411507551481857                    NaN                  NaN   
16463  666407126856765440                    NaN                  NaN   
16464  666396247373291520                    NaN                  NaN   
16465  666373753744588802                    NaN                  NaN   
16466  666362758909284353                    NaN                  NaN   
16467  666353288456101888                    NaN                  NaN   
16468  666345417576210432                    NaN                  NaN   
16469  666337882303524864                    NaN                  NaN   
16470  666293911632134144                    NaN                  NaN   
16471  666287406224695296                    NaN                  NaN   
16472  666273097616637952                    NaN                  NaN   
16473  666268910803644416                    NaN                  NaN   
16474  666104133288665088                    NaN                  NaN   
16475  666102155909144576                    NaN                  NaN   
16476  666099513787052032                    NaN                  NaN   
16477  666094000022159362                    NaN                  NaN   
16478  666082916733198337                    NaN                  NaN   
16479  666073100786774016                    NaN                  NaN   
16480  666071193221509120                    NaN                  NaN   
16481  666063827256086533                    NaN                  NaN   
16482  666058600524156928                    NaN                  NaN   
16483  666057090499244032                    NaN                  NaN   
16484  666055525042405380                    NaN                  NaN   
16

3. Storing and Acting on Wrangling Data

Storing

In [35]:
# save our dataframe in the file specified
tweet_data_clean.to_csv("twitter_archive_master.csv")

In [36]:
# save our dataframe in the file specified
prediction_data_clean.to_csv("twitter_archive_secondary.csv")

Work in progress

Analysis and Insights
3 insights and 1 labeled visualization

In [42]:
#Most common scores on the tweets
tweet_data_clean['score'] = tweet_data_clean['rating_numerator'].astype(str) + "/" + \
                                  tweet_data_clean['rating_denominator'].astype(str)
tweet_data_clean.score.value_counts()

12/10      500
10/10      442
11/10      425
13/10      307
9/10       155
8/10        98
7/10        53
14/10       43
5/10        36
6/10        32
3/10        19
4/10        15
2/10         9
1/10         7
420/10       2
4/20         2
0/10         2
44/40        1
27/10        1
121/110      1
9/11         1
11/15        1
7/11         1
165/150      1
26/10        1
666/10       1
75/10        1
99/90        1
50/50        1
1776/10      1
20/16        1
143/130      1
80/80        1
84/70        1
15/10        1
17/10        1
144/120      1
45/50        1
204/170      1
88/80        1
24/7         1
960/0        1
60/50        1
182/10       1
1/2          1
Name: score, dtype: int64

Insight:
The most common scores are around 10/10 +1 or -1